# Filters

There are four basic types of filters: low-pass, high-pass, band-pass, and band-stop. Each type modifies signals to focus on different ranges of frequencies within them. The plots below demonstrate how frequencies in signals are filtered for each type, presented first with just positive frequencies (easier to understand), then also including negative. Negative frequencies must not be considered because SDRs operate with negative frequencies.

![Alt text](https://pysdr.org/_images/filter_types.png)

![Alt text](https://pysdr.org/_images/tikz-e3aaf5f7c52c87ef9c6da68e8d840ca469e0b53d.svg)

![Alt text](https://pysdr.org/_images/tikz-f88ddd43f673a3ba2436438aa4b2b983f124db7a.svg)

![Alt text](https://pysdr.org/_images/tikz-dc783ab4a1ffd65ac8cb37d787674c288a7566e3.svg)

![Alt text](https://pysdr.org/_images/tikz-76b48577e5a767cb55f861dafb7dcb48c1718257.svg)

An example of creating a filter:

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Odd number of taps for FIR type 1 filter
num_taps = 51 
cut_off = 3000 # Hz
sample_rate = 32000 # Hz
nyq_freq = sample_rate/2.0

# create our low pass filter
# Hamming window is default
h = signal.firwin(num_taps, cut_off/nyq_freq)

# Plot the impulse response
plt.figure(1)
plt.grid(True)
plt.title("Time domain impulse response")
plt.plot(h, '.-')

# Plot the frequency response using FFT
plt.figure(2)
H = np.fft.fftshift(np.fft.fft(h, 1024))
f = np.linspace(-sample_rate/2, sample_rate/2, len(H)) # x axis
plt.grid(True)
plt.title("Frequency reponse using FFT method (linear)")
plt.plot(f, abs(H), '-')

plt.figure(3)
H = np.abs(H[len(H)//2:])
fig, ax = plt.subplots()
w = np.linspace(0, sample_rate/2, len(H))
H_db = 20*np.log10(H)
plt.plot(w, H_db)
plt.grid(True)

axins = inset_axes(ax, width="30%", height="40%")
plt.grid(True)
plt.xlim(2600,3050)
plt.ylim(-10.0, 0)
plt.plot(w, H_db, '-')

# Plot the frequency response using freqz
plt.figure(4)
q, v = signal.freqz(h, worN = 1024)
fig, ax = plt.subplots()
plt.title("Frequency reponse using freqz")
plt.plot((q/np.pi)*nyq_freq, 20*np.log10(abs(v)))
plt.grid(True)

axins = inset_axes(ax, width="30%", height="40%")
plt.grid(True)
plt.xlim(2600,3050)
plt.ylim(-10.0, 0)
plt.plot(w, H_db, '-')
plt.show()

plt.figure(5)
plt.plot((q/np.pi)*nyq_freq, 20*np.log10(abs(v)))
plt.plot(w, H_db)
plt.grid(True)
plt.title("FFT & freqz method give same result?")
plt.show()


# Real vs. Complex Filters

As an example of complex taps, let’s go back to the filtering use-case, except this time we want to receive the other interfering signal (without having to re-tune the radio). That means we want a band-pass filter, but not a symmetrical one. We only want to keep (a.k.a “pass”) frequencies between around 7 kHz to 13 kHz (we don’t want to also pass -13 kHz to -7 kHz):

![Alt text](https://pysdr.org/_images/filter_use_case6.png)

One way to design this kind of filter is to make a low-pass filter with a cutoff of 3 kHz and then frequency shift it. Remember that we can frequency shift x(t) (time domain) by multiplying it by e^{j2\pi f_0t}. In this case f_0 should be 10 kHz, which shifts our filter up by 10 kHz. Recall that in our Python code from above, h was the filter taps of the low-pass filter. In order to create our band-pass filter we just have to multiply those taps by e^{j2\pi f_0t}, although it involves creating a vector to represent time based on our sample period (inverse of the sample rate):